In [1]:
import numpy as np
import pandas as pd

In [2]:
#Collect Data, and mapping 
raw = pd.read_csv('./Podaters/all_individual_differences.csv',encoding ='latin1', low_memory=False)
mapping = pd.read_csv('./Podaters/user_id_mapping.csv', encoding = 'latin1', delimiter=' ', low_memory=False)
mapping.columns = ['new', 'old']
mapping.index = range(1,len(mapping)+1)
top = {'new':[0], 'old':[51]}
orig = pd.DataFrame(data = top)
maps = pd.DataFrame.append(orig, mapping)

In [3]:
#Get all the pips with year4 IDs
yr4ind = ~pd.isnull(raw['userid_yr4'])
yr4 = raw[yr4ind]
print(yr4['user_id'][0:10])

23     51.0
30     63.0
31     65.0
41     93.0
50    116.0
51    121.0
56    131.0
59    140.0
60    142.0
61    144.0
Name: user_id, dtype: float64


In [4]:
#Get all the pips in year4 who are in the mapping 
#And change their ID's to be those in the mapping 
#This is needlessly slow, sry 

# subframe = yr4[yr4['user_id'].isin(maps['old'])]
# subframe.loc[:,'user_id'] = maps['new'][subframe['user_id'].isin(maps['old'])]
subframe = pd.DataFrame(columns = yr4.columns)
for i in range(len(maps)): 
    mask = yr4['userid_yr4'] == maps['old'][i]
    if(np.sum(mask) > 0): 
        keep = yr4[mask]
        keep.loc[:,'user_id'] = maps['new'][i]
        subframe = subframe.append(keep)   

subframe.index = np.arange(len(subframe))
print(subframe.shape)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(915, 1539)


In [5]:
#make array of features to keep: 
keep_feats = \
np.array([
    ['user_id', 0,0], 
    ['gender_yr4', 0,0],
    ['citizen_yr4', 0,0],
    ['workhours_yr4', 0,0],
    ['tradeexp_yr4', 0,0],
    ['tradeenjoy_yr4', 0,0], 
    ['newshrs_yr4', 0,0], 
    ['raven_score_yr4', 0,0], 
    ['crt_score_yr4',0,0], 
    ['crt_?_cor_yr4',1,18],
    ['numeracy_score_yr4', 0,0],
    ['cpk_correct_?', 1, 32],
    ['cpk_score_china'],
    ['cpk_score_globecon'],
    ['cpk_score_iran'],
    ['cpk_score_russia'],
    ['gpk_?',1,13],
    ['gpk_correct_?', 1,13],
    ['gpk_score',0,0], 
    ['aomt_?_yr4',1,9], 
    ['aomt_score_yr4',0,0],
    ['cultwv_ind_score_yr4',0,0], 
    ['cultwv_hier_score_yr4',0,0],
    ['cultwv_ind_?_yr4',1,6], 
    ['cultwv_hier_?_yr4',7,12], 
    ['nfcog_?_yr4',1,18], 
    ['nfcog_score_yr4',0,0], 
    ['memory_math_correct_?',1,6],
    ['memory_word_correct_?',1,6], 
    ['memory_math_score', 0,0],
    ['memory_word_score',0,0], 
    ['helping_score', 0,0], 
    ['orgcommit_?',1,5], 
    ['helping_?', 1,3], 
    ['motivation_?',1,8], 
    ['motivation_intrinsic_score',0,0], 
    ['motivation_prosocial_score',0,0], 
    ['motivation_extrinsic_score',0,0] #,
#     ['age',0,0]
])

tot_feats = []
for i in range(keep_feats.size): 
    if '?' in keep_feats[i][0]: 
#         exp_feat = np.empty(keep_feats[i][2] - keep_feats[i][1] + 1)
        for j in range(keep_feats[i][1], keep_feats[i][2]+1): 
            new_feat = keep_feats[i][0].replace('?', str(j))
            if (subframe.columns.contains(new_feat)): 
                tot_feats.append(new_feat)
            else: 
                print('feature', new_feat, 'is not in subframe')
    else: 
        tot_feats.append(keep_feats[i][0])


In [6]:
mask = subframe.columns.isin(tot_feats)
cleanframe = subframe[subframe.columns[mask]]
print(cleanframe.shape)

(915, 168)


In [7]:
#make all empties 0's 
cleanframe = cleanframe.fillna(0)
cleanframe = cleanframe.replace(0, -1)


In [8]:
cleanframe.to_csv('clean_frame.csv', sep='\t')

In [9]:
test = pd.read_csv('./clean_frame.csv', delimiter='\t', low_memory=False)

In [10]:
print(test.columns.contains('age'))

False


In [11]:
print(np.sum(cleanframe.columns == 'age'))

0


In [12]:
print(cleanframe['newshrs_yr4'][cleanframe['newshares_yr4'] == '8,5'])

KeyError: 'newshares_yr4'